In [1]:
import urllib
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09b_rest.ipynb"
_nb_title = "REST 엔드포인트를 사용한 예측"
_nb_message = ""

### no need to change any of this
_icons=["https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/logo-cloud.png", "https://www.tensorflow.org/images/colab_logo_32px.png", "https://www.tensorflow.org/images/GitHub-Mark-32px.png", "https://www.tensorflow.org/images/download_logo_32px.png"]
_links=["https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?" + urllib.parse.urlencode({"name": _nb_title, "download_url": "https://github.com/ychoi-kr/practical-ml-vision-book-ko/raw/master/"+_nb_loc}), "https://colab.research.google.com/github/ychoi-kr/practical-ml-vision-book-ko/blob/master/{0}".format(_nb_loc), "https://github.com/ychoi-kr/practical-ml-vision-book-ko/blob/master/{0}".format(_nb_loc), "https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/{0}".format(_nb_loc)]
md("""<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="{0}"><img src="{4}"/>AI Platform Notebook에서 실행</a></td><td><a target="_blank" href="{1}"><img src="{5}" />Google Colab에서 실행</a></td><td><a target="_blank" href="{2}"><img src="{6}" />GitHub에서 소스 보기</a></td><td><a href="{3}"><img src="{7}" />노트북 내려받기</a></td></table><br/><br/><h1>{8}</h1>{9}""".format(_links[0], _links[1], _links[2], _links[3], _icons[0], _icons[1], _icons[2], _icons[3], _nb_title, _nb_message))

<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=REST+%EC%97%94%EB%93%9C%ED%8F%AC%EC%9D%B8%ED%8A%B8%EB%A5%BC+%EC%82%AC%EC%9A%A9%ED%95%9C+%EC%98%88%EC%B8%A1&download_url=https%3A%2F%2Fgithub.com%2Fychoi-kr%2Fpractical-ml-vision-book-ko%2Fraw%2Fmaster%2F09_deploying%2F09b_rest.ipynb"><img src="https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/logo-cloud.png"/>AI Platform Notebook에서 실행</a></td><td><a target="_blank" href="https://colab.research.google.com/github/ychoi-kr/practical-ml-vision-book-ko/blob/master/09_deploying/09b_rest.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Google Colab에서 실행</a></td><td><a target="_blank" href="https://github.com/ychoi-kr/practical-ml-vision-book-ko/blob/master/09_deploying/09b_rest.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />GitHub에서 소스 보기</a></td><td><a href="https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/09_deploying/09b_rest.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북 내려받기</a></td></table><br/><br/><h1>REST 엔드포인트를 사용한 예측</h1>

# REST 엔드포인트를 사용한 예측

이 노트북에서는 앞서 훈련 및 저장한 모델(7장)을 가지고 시작한다.
편의상 이 모델을 공개 버킷 [gs://practical-ml-vision-book/flowers_5_trained](gs://practical-ml-vision-book/flowers_5_trained)에 두었다.

이 모델을 REST 엔드포인트로 배포한 다음, POST로 모델을 호출하는 법을 보인다.

## REST 엔드포인트

In [ ]:
!cat ./vertex_deploy.sh

#!/bin/bash

REGION="us-central1"  # make sure you have GPU/TPU quota in this region
ENDPOINT_NAME="flowers_endpoint"
MODEL_NAME="flowers"
MODEL_LOCATION="gs://practical-ml-vision-book/flowers_5_trained"
IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-1:latest"

for i in "$@"
do
case $i in
        -r=*|--region=*) REGION="${i#*=}"; shift ;;
        -e=*|--endpoint_name=*) ENDPOINT_NAME="${i#*=}"; shift ;;
        -m=*|--model_name=*) MODEL_NAME="${i#*=}"; shift ;;
        -l=*|--model_location=*) MODEL_LOCATION="${i#*=}"; shift ;;
        -i=*|--image_uri=*) IMAGE_URI="${i#*=}"; shift ;;
        *) echo "Unknown parameter passed: $1"; exit 1 ;;
esac
done

echo "Deploying model $MODEL_NAME"

if [[ $(gcloud ai endpoints list --region=$REGION --format="value(display_name)" | grep $ENDPOINT_NAME) ]]; then
    echo "The endpoint named $ENDPOINT_NAME already exists."
else
    # Create endpoint.
    echo "Creating $ENDPOINT_NAME endpoint now."
    gcloud ai endpoints create \
    

In [ ]:
!./vertex_deploy.sh

Deploying model flowers
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The endpoint named flowers_endpoint already exists.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The endpoint_id is 4327589805996113920
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Uploading flowers model now.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [8618855115064868864]...done.                            
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The model_id is 4626538221495386112
Deploying model now
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [3921600703717441536]...done.                            
Deployed a model to the endpoint 4327589805996113920. Id of the deployed model: 1963683786742824960.


## 중요: 이 셀을 변경

위의 엔드포인트 ID와 배포된 모델 ID를 기록해 둔다. 아래 셀에 입력한다.

In [ ]:
# 배포한 모델에 맞게 아래를 수정할 것
import os
os.environ['ENDPOINT_ID'] = '4327589805996113920' # 바
os.environ['MODEL_ID'] = '1963683786742824960' # 꿔
os.environ['PROJECT'] = 'ai-analytics-solutions' # 주
os.environ['BUCKET'] = 'ai-analytics-solutions-mlvisionbook' # 세
os.environ['REGION'] = 'us-central1' # 요

## JSON 요청

In [ ]:
%%writefile request.json
{
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Writing request.json


In [ ]:
%%bash
gcloud ai endpoints predict ${ENDPOINT_ID} \
--region=${REGION} \
--json-request=request.json \
--format=json

{
  "deployedModelId": "1963683786742824960",
  "model": "projects/379218021631/locations/us-central1/models/4626538221495386112",
  "modelDisplayName": "flowers",
  "predictions": [
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.619152546
    },
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.999984384
    },
    {
      "flower_type_int": 0,
      "flower_type_str": "daisy",
      "probability": 0.995082855
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.975185812
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.954917
    }
  ]
}


Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]


## HTTP Post를 통해 전송

In [ ]:
# 파이썬에서 호출
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:predict".format(
    REGION, PROJECT, REGION, ENDPOINT_ID)
headers = {"Authorization": "Bearer " + token }
data = {
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{\n  "predictions": [\n    {\n      "flower_type_int": 1,\n      "probability": 0.619152546,\n      "flower_type_str": "dandelion"\n    },\n    {\n      "flower_type_str": "dandelion",\n      "probability": 0.999984384,\n      "flower_type_int": 1\n    },\n    {\n      "probability": 0.995082855,\n      "flower_type_str": "daisy",\n      "flower_type_int": 0\n    },\n    {\n      "probability": 0.975185812,\n      "flower_type_str": "tulips",\n      "flower_type_int": 4\n    },\n    {\n      "flower_type_int": 4,\n      "flower_type_str": "tulips",\n      "probability": 0.954917\n    }\n  ],\n  "deployedModelId": "1963683786742824960",\n  "model": "projects/379218021631/locations/us-central1/models/4626538221495386112",\n  "modelDisplayName": "flowers"\n}\n'


## [선택] CAIP 배치 예측

In [ ]:
%%writefile batchinputs.jsonl
{"filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"}
{"filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"}
{"filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"}
{"filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"}
{"filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"}

In [ ]:
%%bash
gsutil cp batchinputs.jsonl gs://BUCKET

In [ ]:
# 파이썬에서 호출
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']
MODEL_ID = os.environ['MODEL_ID']
BUCKET = os.environ['BUCKET'] # used for staging

BATCH_JOB_NAME = "batch_pred_job"
INPUT_FORMAT = "jsonl"
INPUT_URI = "gs://{}/batchinputs.jsonl".format(BUCKET)
OUTPUT_DIRECTORY = "gs://{}/batch_predictions".format(BUCKET)
MACHINE_TYPE = "n1-standard-2"
STARTING_REPLICA_COUNT = 1
BATCH_SIZE = 64

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/batchPredictionJobs".format(
    REGION, PROJECT, REGION
)
headers = {"Authorization": "Bearer " + token}
data = {
    "displayName": BATCH_JOB_NAME,
    "model": "projects/{}/locations/{}/models/{}".format(
        PROJECT, REGION, MODEL_ID
    ),
    "inputConfig": {
        "instancesFormat": INPUT_FORMAT,
        "gcsSource": {
            "uris": [INPUT_URI],
        },
    },
    "outputConfig": {
        "predictionsFormat": "jsonl",
        "gcsDestination": {
            "outputUriPrefix": OUTPUT_DIRECTORY,
        },
    },
    "dedicatedResources" : {
        "machineSpec" : {
            "machineType": MACHINE_TYPE
        },
        "startingReplicaCount": STARTING_REPLICA_COUNT
    },
    "manualBatchTuningParameters": {
        "batch_size": BATCH_SIZE,
    }
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

## [선택] 아파치 빔에서 온라인 예측을 호출

In [ ]:
import apache_beam as beam
import json
from oauth2client.client import GoogleCredentials
import requests

class ModelPredict:
    def __init__(self, project, region, endpoint_id):
        self._api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:predict".format(
            region, project, region, endpoint_id)   
        
    def __call__(self, filenames):        
        token = GoogleCredentials.get_application_default().get_access_token().access_token
        if isinstance(filenames, str):
            # Only one element, put it into a batch of 1.
            data = {
                "instances": [
                    {"filenames": filenames}
                ]
            }
        else:
            data = {
                "instances": []
            }
            for f in filenames:
                data["instances"].append({
                    "filenames" : f
                })
        # print(data)
        headers = {"Authorization": "Bearer " + token }
        response = requests.post(self._api, json=data, headers=headers)
        response = json.loads(response.content.decode("utf-8"))
        # print(response)
        if isinstance(filenames, str):
            result = response["predictions"][0]
            result["filename"] = filenames
            yield result
        else:
            for (a,b) in zip(filenames, response["predictions"]):
                result = b
                result["filename"] = a
                yield result


PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']

with beam.Pipeline() as p:    
    (p 
     | "input" >> beam.Create([
        "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg",
        "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg",
        "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg",
        "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg",
        "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
     ]) 
     | "batch" >> beam.BatchElements(min_batch_size=2, max_batch_size=3)
     | "addpred" >> beam.FlatMap(ModelPredict(PROJECT, REGION, ENDPOINT_ID))
     | "write" >> beam.Map(print)
    )

## 라이선스
Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.